Factors: A.1.7, A.1.8, A.10, A.2.18(5-year sales growth rank), A.2.19 (Sales Growth)

In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

In [2]:
data = pd.read_csv("merged_final.csv")
#data

In [3]:
data = data[['permno', 'yyyymm', 'monthid', 'ticker', 'cusip', 'RET']]
data

,permno,yyyymm,monthid,ticker,cusip,RET
0,10026,198603.0,75.0,JJSF,466032109,0.133333
1,10026,198604.0,76.0,JJSF,466032109,0.070588
2,10026,198605.0,77.0,JJSF,466032109,0.406593
3,10026,198606.0,78.0,JJSF,466032109,-0.156250
4,10026,198607.0,79.0,JJSF,466032109,-0.375000
...,...,...,...,...,...,...
401583,93429,201802.0,458.0,CBOE,12503M108,-0.166530
401584,93429,201803.0,459.0,CBOE,12503M108,0.021070
401585,93429,201804.0,460.0,CBOE,12503M108,-0.064154
401586,93429,201805.0,461.0,CBOE,12503M108,-0.083817


# Functions

In [4]:
def quantile(df, inVar, outVar, quantCount):
    groups = df.groupby(['monthid'])[inVar]
    df[outVar] = groups.transform(lambda x: pd.qcut(x, q=quantCount, labels=False, duplicates='drop'))
    return df

# Revenue Surprises A.1.7

In [5]:
compustat_rev = pd.read_csv("compustat a-1-7.csv")
#compustat_rev

In [6]:
revDf = compustat_rev[['datadate', 'tic', 'cusip', 'saleq', 'cshprq', 'rdq', 'ajexq']]
revDf["yyyymm"] = pd.DatetimeIndex(revDf['datadate']).year * 100 + pd.DatetimeIndex(revDf['datadate']).month
revDf["monthid"] = (pd.DatetimeIndex(revDf['datadate']).year-1980)*12 + pd.DatetimeIndex(revDf['datadate']).month
revDf['rdqmonthid'] = (pd.DatetimeIndex(revDf['rdq']).year-1980)*12 + pd.DatetimeIndex(revDf['rdq']).month
#revDf

C:\Users\schen\AppData\Local\Temp\ipykernel_12152\2791727363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revDf["yyyymm"] = pd.DatetimeIndex(revDf['datadate']).year * 100 + pd.DatetimeIndex(revDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_12152\2791727363.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revDf["monthid"] = (pd.DatetimeIndex(revDf['datadate']).year-1980)*12 + pd.DatetimeIndex(revDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_12152\2791727363.py

In [7]:
revDf = sqldf("SELECT a.*, b.saleq, b.cshprq, b.ajexq, b.rdq, b.rdqmonthid \
            FROM data as a \
            LEFT JOIN revDf as b \
            ON a.ticker = b.tic and a.monthid > b.rdqmonthid and a.monthid <= b.rdqmonthid + 3")
#revDf

In [8]:
revDf['rev/share'] = revDf['saleq'] / (revDf['cshprq']*revDf['ajexq'])
revDf['revChange'] = revDf['rev/share'].diff(4)
revDf['std'] = revDf['revChange'].rolling(window=8, min_periods=6).std()
revDf['Rs'] = revDf['revChange'] / revDf['std']
#revDf

In [9]:
revDf = revDf[revDf['Rs'] != 0]
revDf = revDf.dropna(subset=['Rs'])

In [10]:
revDf = quantile(revDf, 'Rs', 'Decile', 10)
revDf

,permno,yyyymm,monthid,ticker,cusip,RET,saleq,cshprq,ajexq,rdq,rdqmonthid,rev/share,revChange,std,Rs,Decile
24,10026,198803.0,99.0,JJSF,466032109,0.092308,15.020,3.048,4.0,1988-02-02,98.0,1.231955,0.320481,0.287599,1.114334,5.0
25,10026,198804.0,100.0,JJSF,466032109,0.014085,15.020,3.048,4.0,1988-02-02,98.0,1.231955,-0.375140,0.376739,-0.995755,1.0
26,10026,198805.0,101.0,JJSF,466032109,-0.125000,15.020,3.048,4.0,1988-02-02,98.0,1.231955,-0.375140,0.442909,-0.846991,1.0
27,10026,198805.0,101.0,JJSF,466032109,-0.125000,16.026,3.048,4.0,1988-04-28,100.0,1.314469,-0.292627,0.476625,-0.613956,2.0
28,10026,198806.0,102.0,JJSF,466032109,0.253968,16.026,3.048,4.0,1988-04-28,100.0,1.314469,0.082513,0.476750,0.173074,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419448,93429,201802.0,458.0,CBOE,12503M108,-0.166530,611.400,112.300,1.0,2017-11-07,455.0,5.444346,-0.271979,1.190248,-0.228506,2.0
419449,93429,201803.0,459.0,CBOE,12503M108,0.021070,620.700,112.300,1.0,2018-02-09,458.0,5.527159,-0.189165,1.310804,-0.144312,2.0
419450,93429,201804.0,460.0,CBOE,12503M108,-0.064154,620.700,112.300,1.0,2018-02-09,458.0,5.527159,0.082814,1.354300,0.061149,3.0
419451,93429,201805.0,461.0,CBOE,12503M108,-0.083817,620.700,112.300,1.0,2018-02-09,458.0,5.527159,0.082814,0.977718,0.084701,3.0


# Tax Expense Surprises A.1.8

In [11]:
compustat_tax = pd.read_csv("compustat a-1-8.csv")
#compustat_tax

In [12]:
taxDf = compustat_tax[['datadate', 'tic', 'cusip', 'atq', 'cshprq', 'txtq', 'ajexq']]

In [13]:
taxDf["yyyymm"] = pd.DatetimeIndex(taxDf['datadate']).year * 100 + pd.DatetimeIndex(taxDf['datadate']).month
taxDf["monthid"] = (pd.DatetimeIndex(taxDf['datadate']).year-1980)*12 + pd.DatetimeIndex(taxDf['datadate']).month

C:\Users\schen\AppData\Local\Temp\ipykernel_12152\999723739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxDf["yyyymm"] = pd.DatetimeIndex(taxDf['datadate']).year * 100 + pd.DatetimeIndex(taxDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_12152\999723739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxDf["monthid"] = (pd.DatetimeIndex(taxDf['datadate']).year-1980)*12 + pd.DatetimeIndex(taxDf['datadate']).month


In [14]:
taxDf = sqldf("SELECT a.*, b.atq, b.cshprq, b.txtq, b.ajexq \
            FROM data as a \
            LEFT JOIN taxDf as b \
            ON a.ticker = b.tic and a.monthid >= b.monthid + 4 and a.monthid <= b.monthid + 6")

In [15]:
taxDf['taxExp'] = taxDf['txtq'] / (taxDf['cshprq'] * taxDf['ajexq'])
taxDf['assets/share'] = taxDf['atq'] / (taxDf['cshprq'] * taxDf['ajexq'])
taxDf['tes'] = (taxDf['taxExp'] - taxDf['taxExp'].shift(12)) * taxDf['assets/share'].shift(12)
taxDf = taxDf[taxDf['tes'] != 0]
taxDf = taxDf.dropna(subset=['tes'])

In [16]:
taxDf = quantile(taxDf, 'tes', 'Decile', 10)
taxDf

C:\Users\schen\anaconda3\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\schen\anaconda3\lib\site-packages\numpy\lib\function_base.py:4488: RuntimeWarning: invalid value encountered in multiply
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))


,permno,yyyymm,monthid,ticker,cusip,RET,atq,cshprq,txtq,ajexq,taxExp,assets/share,tes,Decile
12,10026,198703.0,87.0,JJSF,466032109,0.030769,23.286,2.918,0.311,4.0,0.026645,1.995031,-0.022394,2.0
16,10026,198707.0,91.0,JJSF,466032109,0.114504,25.584,2.985,0.402,4.0,0.033668,2.142714,0.027242,5.0
17,10026,198708.0,92.0,JJSF,466032109,0.123288,25.584,2.985,0.402,4.0,0.033668,2.142714,0.027242,5.0
18,10026,198709.0,93.0,JJSF,466032109,0.060976,25.584,2.985,0.402,4.0,0.033668,2.142714,0.027242,6.0
19,10026,198710.0,94.0,JJSF,466032109,-0.379310,46.717,2.985,0.415,4.0,0.034757,3.912647,0.033756,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401802,93429,201802.0,458.0,CBOE,12503M108,-0.166530,5228.100,112.300,45.600,1.0,0.406055,46.554764,0.407873,5.0
401803,93429,201803.0,459.0,CBOE,12503M108,0.021070,5228.100,112.300,45.600,1.0,0.406055,46.554764,0.407873,5.0
401804,93429,201804.0,460.0,CBOE,12503M108,-0.064154,5265.700,112.300,-153.000,1.0,-1.362422,46.889581,-10.138196,2.0
401805,93429,201805.0,461.0,CBOE,12503M108,-0.083817,5265.700,112.300,-153.000,1.0,-1.362422,46.889581,-10.138196,3.0


# A.1.10

In [17]:
compustat_earnings = pd.read_csv("compustat a-1-10.csv")
#compustat_earnings

In [18]:
earnDf = compustat_earnings[['datadate', 'tic', 'cusip', 'rdq', 'ibq']]
earnDf["yyyymm"] = pd.DatetimeIndex(earnDf['datadate']).year * 100 + pd.DatetimeIndex(earnDf['datadate']).month
earnDf["monthid"] = (pd.DatetimeIndex(earnDf['datadate']).year-1980)*12 + pd.DatetimeIndex(earnDf['datadate']).month
earnDf['rdqmonthid'] = (pd.DatetimeIndex(earnDf['rdq']).year-1980)*12 + pd.DatetimeIndex(earnDf['rdq']).month

C:\Users\schen\AppData\Local\Temp\ipykernel_12152\428864831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earnDf["yyyymm"] = pd.DatetimeIndex(earnDf['datadate']).year * 100 + pd.DatetimeIndex(earnDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_12152\428864831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earnDf["monthid"] = (pd.DatetimeIndex(earnDf['datadate']).year-1980)*12 + pd.DatetimeIndex(earnDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_12152\428864831

In [19]:
earnDf = sqldf("SELECT a.*, b.ibq, b.rdq, b.rdqmonthid \
            FROM data as a \
            LEFT JOIN earnDf as b \
            ON a.ticker = b.tic and a.monthid > b.rdqmonthid and a.monthid <= b.rdqmonthid + 3")
#earnDf

In [20]:
earnDf = earnDf.dropna(subset=['ibq'])
earnDf.reset_index(drop=True, inplace=True)
#earnDf

In [21]:
earnDf['Nei'] = 0
#earnDf


In [22]:
for i in range(len(earnDf)):
    if i - 12 >= 0 and earnDf.loc[i-12, 'ticker'] == earnDf.loc[i, 'ticker'] and earnDf.loc[i-12, 'ibq'] < earnDf.loc[i, 'ibq']:
        earnDf.loc[i, 'Nei'] = min(earnDf.loc[i-12, 'ibq'] + 1, 8)
earnDf

,permno,yyyymm,monthid,ticker,cusip,RET,ibq,rdq,rdqmonthid,Nei
0,10026,198705.0,89.0,JJSF,466032109,-0.042373,0.533,1987-04-28,88.0,0.0
1,10026,198706.0,90.0,JJSF,466032109,0.159292,0.533,1987-04-28,88.0,0.0
2,10026,198707.0,91.0,JJSF,466032109,0.114504,0.533,1987-04-28,88.0,0.0
3,10026,198709.0,93.0,JJSF,466032109,0.060976,0.803,1987-08-05,92.0,0.0
4,10026,198710.0,94.0,JJSF,466032109,-0.379310,0.803,1987-08-05,92.0,0.0
...,...,...,...,...,...,...,...,...,...,...
348490,93429,201802.0,458.0,CBOE,12503M108,-0.166530,60.300,2017-11-07,455.0,8.0
348491,93429,201803.0,459.0,CBOE,12503M108,0.021070,257.100,2018-02-09,458.0,8.0
348492,93429,201804.0,460.0,CBOE,12503M108,-0.064154,257.100,2018-02-09,458.0,8.0
348493,93429,201805.0,461.0,CBOE,12503M108,-0.083817,257.100,2018-02-09,458.0,8.0


# A.2.18 & A.2.19

In [23]:
compustat_sales = pd.read_csv("compustat annual sales.csv")

In [24]:
salesDf = compustat_sales[['datadate', 'tic', 'cusip', 'sale']]
salesDf["yyyymm"] = pd.DatetimeIndex(salesDf['datadate']).year * 100 + pd.DatetimeIndex(salesDf['datadate']).month
salesDf["monthid"] = (pd.DatetimeIndex(salesDf['datadate']).year-1980)*12 + pd.DatetimeIndex(salesDf['datadate']).month
#salesDf

C:\Users\schen\AppData\Local\Temp\ipykernel_12152\1244776288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salesDf["yyyymm"] = pd.DatetimeIndex(salesDf['datadate']).year * 100 + pd.DatetimeIndex(salesDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_12152\1244776288.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salesDf["monthid"] = (pd.DatetimeIndex(salesDf['datadate']).year-1980)*12 + pd.DatetimeIndex(salesDf['datadate']).month


In [25]:
salesDf = sqldf("SELECT b.*, a.sale, a.monthid as salemonthid \
                FROM salesDf as a \
                LEFT JOIN data as b \
                ON a.tic = b.ticker and a.monthid + 1 = b.monthid")
#salesDf

In [26]:
salesDf['salesgrowth'] = (salesDf['sale'] / salesDf['sale'].shift(1)) - 1
salesDf.loc[salesDf['ticker'] != salesDf['ticker'].shift(1), 'salesgrowth'] = 0
salesDf = salesDf.dropna(subset=['ticker'])
salesDf = salesDf.loc[salesDf['sale'] >= 0]
#salesDf

In [ ]:
salesDf = quantile(salesDf, 'salesgrowth', 'annual decile', 10)
#salesDf

C:\Users\schen\anaconda3\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [ ]:
# Sr is 5 year annual sales growth rank
salesDf.reset_index(drop=True, inplace=True)
salesDf['Sr'] = 0
for i in range(len(salesDf)):
    if i - 5 >= 0 and salesDf.loc[i-5, 'ticker'] == salesDf.loc[i, 'ticker']:
        numerator = 0
        for j in range(1, 6):
            numerator += (6 - j) * salesDf.loc[i-j, 'annual decile']
        denominator = 15
        salesDf.loc[i, 'Sr'] = numerator/denominator
#salesDf.head(10)

In [ ]:
salesDf = quantile(salesDf, 'Sr', '5 year decile', 10)
salesDf.head(10)